In [ ]:
# Deep learning
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

# Miscelanous
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Dataset Path
TRAIN_PATH = "/kaggle/input/intel-image-classification/seg_train/seg_train"
TEST_PATH = "/kaggle/input/intel-image-classification/seg_test/seg_test"
PRED_PATH = "/kaggle/input/intel-image-classification/seg_pred/seg_pred"

In [ ]:
# device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Preparation

In [ ]:
label_mapping = {
    'buildings': 0,
    'forest':1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5 
}

In [ ]:
for category in os.listdir(TRAIN_PATH):
    category_dir = os.path.join(TRAIN_PATH, category)
    print(category_dir)

## Read dataset with process bar from tqdm

In [ ]:
def read_data(path):
    X = []
    y = []
    for category in os.listdir(path):
        category_dir = os.path.join(path, category)
        for img_file in tqdm(os.listdir(category_dir)):
            img_path = os.path.join(category_dir, img_file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (150,150))
            # convert img to torch tensor
            transform = transforms.ToTensor()
            img = transform(img)
            X.append(img)
            y.append(label_mapping[category])
            
    y = torch.tensor(y)
    return (X,y)

In [ ]:
X_train, y_train = read_data(TRAIN_PATH)

## Pytorch Custom Dataset

In [ ]:
class IntelDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.images = X
        self.labels = y
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        return self.images[idx],self.labels[idx]

In [ ]:
train_dataset = IntelDataset(X_train, y_train)

## Dataloader

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

# Modeling

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_channels, num_classes=6):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(in_features=37*37*64, out_features=500)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=500, out_features=num_classes)
        self.logsoftmax = nn.LogSoftmax(dim=1)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.view(x.size(0), -1) # flatten the features map
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        out = self.logsoftmax(x)
        return out

In [ ]:
# Initialize Model
model = CNN(num_channels=3).to(device)

In [ ]:
# Optimizer and Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

## Training

In [ ]:
for epoch in range(10):
    model.train()
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backpropagation and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%110==0:
            print (f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Testing

In [ ]:
X_test, y_test = read_data(TEST_PATH)

In [ ]:
test_dataset = IntelDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False
)

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        # predict
        preds = model(images)
        # get the label
        _, predicted = torch.max(preds.data, 1)
        # num of images
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Test Accuracy of the model on the test images: {100 * correct / total}%')

In [ ]:
# # save the model
# torch.save(model.state_dict(), 'model.ckpt')